# NFL Baseline
- create target_df (distance in tracking_df is lower than threshold=3)
https://www.kaggle.com/code/stgkrtua/nfl-creatatraindataset-targetdf
- create dataset save frames in target_df
https://www.kaggle.com/code/stgkrtua/nfl-createdataset-saveframes
- check saved images
https://www.kaggle.com/code/stgkrtua/nfl-checkdataset-plotsavedimage

# import libraries

In [1]:
# general
import os
import gc
import pickle
import glob
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import cv2
import matplotlib.pyplot as plt
import time
import math

import sys
sys.path.append('/kaggle/input/timm-pytorch-image-models/pytorch-image-models-master')
import timm


# deep learning
from torch.utils.data import Dataset, DataLoader
from torch.optim import SGD, Adam, AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR, CosineAnnealingWarmRestarts, ReduceLROnPlateau
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import albumentations as A
from albumentations.pytorch import ToTensorV2

from sklearn.model_selection import GroupKFold

# loss metrics
from sklearn.metrics import matthews_corrcoef, confusion_matrix, roc_auc_score

# import cudf
import polars as pl

import mlflow
# import wandb
# warningの表示方法の設定
import warnings
warnings.filterwarnings("ignore")

# Set Configurations

In [2]:
CFG = {
        "kaggle" : False,
        "DEBUG" : False,
        # model config
        "num_track_features" : 3,

        "features" : ['x_position_1', 'y_position_1', 'x_position_2', 'y_position_2', 
                      'speed_1', 'distance_1', 'direction_1', 'orientation_1','acceleration_1', 'sa_2', 
                      'speed_2', 'distance_2', 'direction_2', 'orientation_2', 'acceleration_2', 'sa_2',
                      'players_dis', #'is_ground'
                    #   'speed_diff', 'distance_diff', 'direction_diff', 'orientation_diff','acceleration_diff', 'sa_diff', # diff-feature
                    #   'x_move_distance_2', 'y_move_distance_1', 'x_move_distance_2', 'y_move_distance_2',
                    #   'x_position_diff_rollsum','y_position_diff_rollsum', 
                    #   'speed_diff_rollsum','distance_diff_rollsum', 'direction_diff_rollsum','orientation_diff_rollsum', 
                    #   'acceleration_diff_rollsum', 'sa_diff_rollsum','players_dis_rollsum', 
                      ],
        "SHIFT_COLS" : ["x_position", "y_position", 'speed', 'distance', 'direction', 'orientation', 'acceleration', 'sa',],
        "SHIFT_NUM" : range(-6,6,1),
                
        "xpos_1" : ['x_position_shift-6_1', 'x_position_shift-5_1', 'x_position_shift-4_1', 'x_position_shift-3_1', 'x_position_shift-2_1', 'x_position_shift-1_1',
                 'x_position_shift0_1', 'x_position_shift1_1', 'x_position_shift2_1', 'x_position_shift3_1', 'x_position_shift4_1', 'x_position_shift5_1'],
        "ypos_1" : ['y_position_shift-6_1', 'y_position_shift-5_1', 'y_position_shift-4_1', 'y_position_shift-3_1', 'y_position_shift-2_1', 'y_position_shift-1_1', 
                 'y_position_shift0_1', 'y_position_shift1_1', 'y_position_shift2_1', 'y_position_shift3_1', 'y_position_shift4_1', 'y_position_shift5_1'],
        "speed_1" : ['speed_shift-6_1', 'speed_shift-5_1', 'speed_shift-4_1', 'speed_shift-3_1', 'speed_shift-2_1', 'speed_shift-1_1', 
                 'speed_shift0_1', 'speed_shift1_1', 'speed_shift2_1', 'speed_shift3_1', 'speed_shift4_1', 'speed_shift5_1'],
        "dire_1" : ['direction_shift-6_1', 'direction_shift-5_1', 'direction_shift-4_1', 'direction_shift-3_1', 'direction_shift-2_1', 'direction_shift-1_1',
                 'direction_shift0_1', 'direction_shift1_1', 'direction_shift2_1', 'direction_shift3_1', 'direction_shift4_1', 'direction_shift5_1'],
        "orie_1" : ['orientation_shift-6_1', 'orientation_shift-5_1', 'orientation_shift-4_1', 'orientation_shift-3_1', 'orientation_shift-2_1', 'orientation_shift-1_1',
                 'orientation_shift0_1', 'orientation_shift1_1', 'orientation_shift2_1', 'orientation_shift3_1', 'orientation_shift4_1', 'orientation_shift5_1'],
        "acc_1" : ['acceleration_shift-6_1', 'acceleration_shift-5_1', 'acceleration_shift-4_1', 'acceleration_shift-3_1', 'acceleration_shift-2_1', 'acceleration_shift-1_1',
                 'acceleration_shift0_1', 'acceleration_shift1_1', 'acceleration_shift2_1', 'acceleration_shift3_1', 'acceleration_shift4_1', 'acceleration_shift5_1'],
        "sa_1" : ['sa_shift-6_1', 'sa_shift-5_1', 'sa_shift-4_1', 'sa_shift-3_1', 'sa_shift-2_1', 'sa_shift-1_1', 
                 'sa_shift0_1', 'sa_shift1_1', 'sa_shift2_1', 'sa_shift3_1', 'sa_shift4_1', 'sa_shift5_1'],
        "dis_1" : ['distance_shift-6_1', 'distance_shift-5_1', 'distance_shift-4_1', 'distance_shift-3_1', 'distance_shift-2_1', 'distance_shift-1_1',
                 'distance_shift0_1', 'distance_shift1_1', 'distance_shift2_1', 'distance_shift3_1', 'distance_shift4_1', 'distance_shift5_1'],
        
        
        "xpos_2" : ['x_position_shift-6_2', 'x_position_shift-5_2', 'x_position_shift-4_2', 'x_position_shift-3_2', 'x_position_shift-2_2', 'x_position_shift-1_2',
                  'x_position_shift0_2', 'x_position_shift1_2', 'x_position_shift2_2', 'x_position_shift3_2', 'x_position_shift4_2', 'x_position_shift5_2'],
        "ypos_2" : ['y_position_shift-6_2', 'y_position_shift-5_2', 'y_position_shift-4_2', 'y_position_shift-3_2', 'y_position_shift-2_2', 'y_position_shift-1_2',
                 'y_position_shift0_2', 'y_position_shift1_2', 'y_position_shift2_2', 'y_position_shift3_2', 'y_position_shift4_2', 'y_position_shift5_2'],
        "speed_2" : ['speed_shift-6_2', 'speed_shift-5_2', 'speed_shift-4_2', 'speed_shift-3_2', 'speed_shift-2_2', 'speed_shift-1_2', 
                 'speed_shift0_2', 'speed_shift1_2', 'speed_shift2_2', 'speed_shift3_2', 'speed_shift4_2', 'speed_shift5_2'],
        "dire_2" : ['direction_shift-6_2', 'direction_shift-5_2', 'direction_shift-4_2', 'direction_shift-3_2', 'direction_shift-2_2', 'direction_shift-1_2',
                 'direction_shift0_2', 'direction_shift1_2', 'direction_shift2_2', 'direction_shift3_2', 'direction_shift4_2', 'direction_shift5_2'],
        "orie_2" : ['orientation_shift-6_2', 'orientation_shift-5_2', 'orientation_shift-4_2', 'orientation_shift-3_2', 'orientation_shift-2_2', 'orientation_shift-1_2',
                 'orientation_shift0_2', 'orientation_shift1_2', 'orientation_shift2_2', 'orientation_shift3_2', 'orientation_shift4_2', 'orientation_shift5_2'],
        "acc_2" : ['acceleration_shift-6_2', 'acceleration_shift-5_2', 'acceleration_shift-4_2', 'acceleration_shift-3_2', 'acceleration_shift-2_2', 'acceleration_shift-1_2',
                 'acceleration_shift0_2', 'acceleration_shift1_2', 'acceleration_shift2_2', 'acceleration_shift3_2', 'acceleration_shift4_2', 'acceleration_shift5_2'],
        "sa_2" : ['sa_shift-6_2', 'sa_shift-5_2', 'sa_shift-4_2', 'sa_shift-3_2', 'sa_shift-2_2', 'sa_shift-1_2',
                 'sa_shift0_2', 'sa_shift1_2', 'sa_shift2_2', 'sa_shift3_2', 'sa_shift4_2', 'sa_shift5_2'],
        "dis_2" : ['distance_shift-6_2', 'distance_shift-5_2', 'distance_shift-4_2', 'distance_shift-3_2', 'distance_shift-2_2', 'distance_shift-1_2',
                 'distance_shift0_2', 'distance_shift1_2', 'distance_shift2_2', 'distance_shift3_2', 'distance_shift4_2', 'distance_shift5_2'],
                
                
                
        # learning config
        "n_epoch" : 10,
        "n_folds": 3,
        "train_folds" : [0,1,2],
        "lr" : 5e-3,
        "T_max" : 10,
        "min_lr" : 1e-8,
        "weight_decay" : 1e-6,

        # etc
        "print_freq" : 1000,
        "random_seed" : 21,

        # data config    
        "batch_size" : 256,
        "num_workers" : 2,
        "sample_num" : -1,

        "EXP_CATEGORY" : "exps_conv1d",
        "EXP_NAME" : "expT003_Trackshift1D",
}

if CFG["DEBUG"]:
    CFG["EXP_CATEGORY"] = "DEBUG"
    CFG["EXP_NAME"] = "DEBUG"
    CFG["n_epoch"] = 2
    CFG["sample_num"] = 1000
    CFG["batch_size"] = 32
    CFG["train_folds"] : [0,1]


CFG["INPUT_DIR"] = "/workspace/input"
CFG["OUTPUT_DIR"] = "/workspace/output"
CFG["TRAIN_HELMET_CSV"] = os.path.join(CFG["INPUT_DIR"], "train_baseline_helmets.csv")
CFG["TRAIN_TRACKING_CSV"] = os.path.join(CFG["INPUT_DIR"], "train_player_tracking.csv")
CFG["TRAIN_VIDEO_META_CSV"] = os.path.join(CFG["INPUT_DIR"], "train_video_metadata.csv")
CFG["TRAIN_LABEL_CSV"] = os.path.join(CFG["INPUT_DIR"], "train_labels.csv")
CFG["SAVED_CONTACT_CSV"] = os.path.join(CFG["INPUT_DIR"], "Saved_contact_frames.csv")
CFG["CONTACT_IMG_DIR"] = os.path.join(CFG["INPUT_DIR"], "contact_images")
CFG["MODEL_DIR"] = os.path.join(CFG["OUTPUT_DIR"], CFG["EXP_NAME"] ,"model")
    
if not CFG["kaggle"] and not CFG["DEBUG"]:
    os.mkdir(os.path.join(CFG["OUTPUT_DIR"], CFG["EXP_NAME"]))
    os.mkdir(CFG["MODEL_DIR"])


In [3]:
if CFG["kaggle"]:
    os.environ["WANDB_SILENT"] = "true"
    WANDB_CONFIG = {'competition': 'NFL', '_wandb_kernel': 'taro'}
    # Secrets
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    secret_value_0 = user_secrets.get_secret("wandb")

    !wandb login $secret_value_0
    #! TODO : logger settings
    wandb.init(project=WANDB_CONFIG["competition"], config=CFG, group=CFG["EXP_CATEGORY"], name=CFG["EXP_NAME"])

else:
    mlflow.set_tracking_uri("/workspace/mlruns")
    experiment = mlflow.get_experiment_by_name(CFG["EXP_CATEGORY"])
    if experiment is None:  # 当該Experiment存在しないとき、新たに作成
        experiment_id = mlflow.create_experiment(name=CFG["EXP_CATEGORY"])
    else: # 当該Experiment存在するとき、IDを取得
        experiment_id = experiment.experiment_id

# Utils

In [4]:
def logging_metrics_epoch(fold, epoch, train_loss_avg, valid_loss_avg, score, threshold, tn_best, fp_best, fn_best, tp_best, auc_score):
    if CFG["kaggle"]:
        wandb.log({"loss avg":{f"train/fold{fold}": train_loss_avg,
                                f"valid/fold{fold}": valid_loss_avg}}, step=epoch)
        wandb.log({"Metircs" : {f"score/fold{fold}":score,
                                f"score threshold/fold{fold}":threshold,
                                f"tn/fold{fold}":tn_best,
                                f"fp/fold{fold}":fp_best,
                                f"fn/fold{fold}":fn_best,
                                f"tp/fold{fold}":tp_best,
                                f"auc/fold{fold}":auc_score,
                               }}, step=epoch)
    else:
        mlflow.log_metric(f"fold{fold} train loss avg", train_loss_avg, step=epoch)
        mlflow.log_metric(f"fold{fold} valid loss avg", valid_loss_avg, step=epoch)
        mlflow.log_metric(f"fold{fold} score", score, step=epoch)
        mlflow.log_metric(f"fold{fold} score threshold", threshold, step=epoch)
        mlflow.log_metric(f"fold{fold} tn", tn_best, step=epoch)
        mlflow.log_metric(f"fold{fold} fp", fp_best, step=epoch)
        mlflow.log_metric(f"fold{fold} fn", fn_best, step=epoch)
        mlflow.log_metric(f"fold{fold} tp", tp_best, step=epoch)
        mlflow.log_metric(f"fold{fold} auc", auc_score, step=epoch)

In [5]:
def seed_everything(seed=CFG["random_seed"]):
    #os.environ['PYTHONSEED'] = str(seed)
    np.random.seed(seed%(2**32-1))
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic =True
    torch.backends.cudnn.benchmark = False
seed_everything()

# device optimization
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(f'Using device: {device}')

Using device: cuda


In [6]:
def asMinutes(s):
    """Convert Seconds to Minutes."""
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    """Accessing and Converting Time Data."""
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))

class AverageMeter(object):
    """Computes and stores the average and current value."""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

# Dataset Functions

In [7]:
def add_feature_cols(df_, FEATURE_COLS,remove_col_list):
    additional_cols = list(df_.columns)
    additional_cols = [col for col in additional_cols if not col in remove_col_list]
    FEATURE_COLS.extend(additional_cols)
    return FEATURE_COLS

## target df func

In [8]:
# -------------------------------
# replace polars
# -------------------------------
def create_trackmerged_ftr(target_df, FEATURE_COLS,
                           diff_cols = ['x_position', 'y_position', 'speed', 'distance',
                                        'direction', 'orientation', 'acceleration', 'sa']):
    target_df = target_df.with_column((np.sqrt(np.square(pl.col("x_position_1")-pl.col("x_position_2")) \
                                              + np.square(pl.col("y_position_1")-pl.col("y_position_2"))) \
                                      ).alias("players_dis"))
    target_df = target_df.with_column(pl.col("players_dis").fill_null(0))
    
    # players distance sum(in shift range time : default(-6~6 frames not step))
    players_distance_sum = 0
    for idx in range(-6,6,1):
        players_distance_sum += np.sqrt((target_df[f"x_position_shift{idx}_1"] - target_df[f"x_position_shift{idx}_2"])**2 \
                                       + (target_df[f"y_position_shift{idx}_1"] - target_df[f"y_position_shift{idx}_2"])**2)
    target_df = target_df.with_column(pl.Series("players_distance_sum", players_distance_sum))
    target_df = target_df.with_column(pl.col("players_distance_sum").fill_null(0))
    FEATURE_COLS.append("players_distance_sum")

    # players each axis distance sum(in shift range time : default(-6~6 frames not step))
    for axis in ["x", "y"]:
        axis_distance_1 = 0
        axis_distance_2 = 0
        for idx in range(-6, 5, 1):
            axis_distance_1 += abs(target_df[f"{axis}_position_shift{idx}_1"] - target_df[f"{axis}_position_shift{idx+1}_1"])
            axis_distance_2 += abs(target_df[f"{axis}_position_shift{idx}_2"] - target_df[f"{axis}_position_shift{idx+1}_2"])
        target_df = target_df.with_column(pl.Series(f"{axis}_move_distance_1", axis_distance_1))
        target_df = target_df.with_column(pl.col(f"{axis}_move_distance_1").fill_null(0))
        target_df = target_df.with_column(pl.Series(f"{axis}_move_distance_2", axis_distance_2))
        target_df = target_df.with_column(pl.col(f"{axis}_move_distance_2").fill_null(0))
        FEATURE_COLS.extend([f"{axis}_move_distance_1", f"{axis}_move_distance_2"])

    # players difference ftr (in each step)
    for col in diff_cols:
        colname = f"{col}_diff"
        target_df = target_df.with_column((abs(pl.col(f"{col}_1") - pl.col(f"{col}_2"))).alias(colname))
        target_df = target_df.with_column(pl.col(colname).fill_null(0))
        FEATURE_COLS.append(colname)
        
    return target_df, FEATURE_COLS

In [9]:
# -------------------------------
# replace polars
# polars groupby rolling (groupby_dynamics)
# https://pola-rs.github.io/polars/py-polars/html/reference/dataframe/api/polars.DataFrame.groupby_dynamic.html
# -------------------------------
def create_roll_ftr(target_df, FEATURE_COLS_,
                    key_cols = ["contact_id", "game_play", "nfl_player_id_1", 
                                "nfl_player_id_2", "datetime", "step"]):
    roll_df = target_df.select(roll_cols+key_cols)

    roll_df = roll_df.with_column(pl.concat_str([pl.col("game_play"),
                                                 pl.col("nfl_player_id_1"),
                                                 pl.col("nfl_player_id_2"),
                                                ], sep='_').alias('key'))
    # cast datetime
    roll_df = roll_df.with_column(pl.col('datetime').str.strptime(pl.Datetime,
                                                                          fmt="%+",
                                                                          strict=False
                                                                         ).alias('datetime'))
    # groupby rolling
    roll_df = roll_df.groupby_dynamic("step", every="1i", period="6i", by="key", closed="both").agg([pl.col(roll_cols).sum().suffix("_rollsum"), pl.col("contact_id")])
    roll_df = roll_df.with_column(pl.col("contact_id").apply(lambda x:x[0]))
    roll_df = roll_df.drop(["key", "step"])
    target_df = target_df.join(roll_df, on="contact_id", how="left")
    additional_cols = [col+"_rollsum" for col in roll_cols]
    FEATURE_COLS_.extend(additional_cols)
    return target_df, FEATURE_COLS_

In [10]:
# -------------------------------
# replace polars
# -------------------------------
def create_helmetmerged_ftr(target_df, FEATURE_COLS_):
    # helmet center distance feature
    for view in ["Endzone", "Sideline"]:
        for p_id in ["1", "2"]: 
            # get helmet center
            target_df = target_df.with_column((pl.col(f"{view[0]}_left_{p_id}") + (pl.col(f"{view[0]}_width_{p_id}")//2)).alias(f"{view[0]}_Wcenter_{p_id}"))
            target_df = target_df.with_column((pl.col(f"{view[0]}_top_{p_id}") + (pl.col(f"{view[0]}_height_{p_id}")//2)).alias(f"{view[0]}_Hcenter_{p_id}"))
        # helmet center distance
        target_df = target_df.with_column((np.sqrt(np.square(pl.col(f"{view[0]}_Wcenter_1") - pl.col(f"{view[0]}_Wcenter_2")) \
                                                  + np.square(pl.col(f"{view[0]}_Hcenter_1") - pl.col(f"{view[0]}_Hcenter_2")))
                                          ).alias(f"{view[0]}_helmet_dis"))

        # GがNanになるので0にしておく
        target_df = target_df.with_column(pl.col(f"{view[0]}_helmet_dis").fill_null(0))
        FEATURE_COLS_.append(f"{view[0]}_helmet_dis")
    
    # helmet cols fillna(0) after get helmet distance 
    helmet_cols = ['E_left_1', 'E_width_1', 'E_top_1', 'E_height_1',
                   'E_left_2', 'E_width_2', 'E_top_2', 'E_height_2', 
                   'S_left_1','S_width_1', 'S_top_1', 'S_height_1', 
                   'S_left_2', 'S_width_2', 'S_top_2', 'S_height_2']
    target_df = target_df.with_column(pl.col(helmet_cols).fill_null(0))

    return target_df, FEATURE_COLS_

In [11]:
# -------------------------------
# replace polars
# -------------------------------
def get_categorical_ftr(target_df, FEATURE_COLS_):
    target_df = target_df.with_column((pl.col("nfl_player_id_2")== "G").cast(int).alias("is_ground"))
    
#     target_df["nfl_player_id_2"] = target_df["nfl_player_id_2"].replace("G", "99999").astype(np.int64) # when inference this is after cnn pred
    target_df = target_df.with_column((1 - ((pl.col("E_width_1")==0) & (pl.col("E_width_2")==0) \
                                          & (pl.col("S_width_1")==0) & (pl.col("S_width_2")==0)).cast(int)).alias("is_helmet"))
    target_df = target_df.with_column((1 - ((pl.col("E_width_1")==0) & (pl.col("E_width_2")==0)).cast(int)).alias("is_E_helmet"))
    target_df = target_df.with_column((1 - ((pl.col("S_width_1")==0) & (pl.col("S_width_2")==0)).cast(int)).alias("is_S_helmet"))
    target_df = target_df.with_column(((pl.col("is_E_helmet")==1) & (pl.col("is_S_helmet")==1)).cast(int).alias("both_helmet"))
    # set team 
    target_df = target_df.with_column(((pl.col("team_1")=="home").cast(int)).alias("team_1"))
    target_df = target_df.with_column(((pl.col("team_2")=="home").cast(int)).alias("team_2"))
    
    target_df = target_df.fill_null(0)
    target_df = target_df.fill_nan(0)
    FEATURE_COLS_.extend(["is_ground", "is_helmet"])
    return target_df, FEATURE_COLS_

## tracking df func

In [12]:
# -------------------------------
# replace polars
# -------------------------------
def get_tracking_shift(tracking_df_, shift_cols=CFG["SHIFT_COLS"], shift_nums=CFG["SHIFT_NUM"]):
    # get shift key
    #     tracking_df = tracking_df.with_column((pl.col('step')/10*59.94+5*59.94 + 5000).alias('frame_add'))
    tracking_df_ = tracking_df_.with_column(pl.concat_str([pl.col("game_play"),
                                                         pl.col("nfl_player_id"),
                                                        ], sep='_').alias('shift_key'))
    # get shift features
    SHIFT_COLS_ = []
    for num in shift_nums:
        tracking_df_ = tracking_df_.with_columns(
                            pl.col(shift_cols).shift(periods=num).over("shift_key").suffix(f"_shift{num}"))
    for col in shift_cols:
        colname = [f"{col}_shift{idx}" for idx in shift_nums]
        SHIFT_COLS_.extend(colname)
                    
    return tracking_df_, SHIFT_COLS_

In [13]:
# -------------------------------
# replace polars
# -------------------------------
def target_merge_tracking(target_df, tracking_df, FEATURE_COLS_, SHIFT_COLS_,
                          TRACKING_COLS_ = ["game_play", "nfl_player_id", "step", 
                                           "x_position", "y_position", "datetime",
                                           "speed","distance","direction","orientation",
                                           "acceleration","sa", "team", "jersey_number"]):
    target_df = target_df.with_column(pl.concat_str([pl.col("game_play"),
                                                     pl.col("step").cast(str),
                                                     pl.col("nfl_player_id_1"),
                                                    ], sep='_').alias('game_step_player_1'))
    target_df = target_df.with_column(pl.concat_str([pl.col("game_play"),
                                                     pl.col("step").cast(str),
                                                     pl.col("nfl_player_id_2"),
                                                    ], sep='_').alias('game_step_player_2'))

    TRACKING_COLS_.extend(SHIFT_COLS_)
    # print(TRACKING_COLS_)
    tracking_df = tracking_df.select(TRACKING_COLS_)
    tracking_df = tracking_df.with_column(pl.concat_str([pl.col("game_play"),
                                                         pl.col("step").cast(str),
                                                         pl.col("nfl_player_id"),
                                                        ], sep='_').alias('game_step_player'))

    tracking_df = tracking_df.drop(["game_play", "step", "nfl_player_id", "datetime"])

    # merge tracking to target
    for player_id in [1,2]:
        tracking_player = tracking_df.select([pl.all().suffix(f"_{player_id}")])
        target_df = target_df.join(tracking_player, on=[f"game_step_player_{player_id}"], how="left")
        # add features col
        FEATURE_COLS_ = add_feature_cols(tracking_player, FEATURE_COLS_,
                                        [f"game_step_player_{player_id}", f"frame_{player_id}", f"datetime_{player_id}"])
    # drop col
    target_df = target_df.drop(["game_step_player_1", "game_step_player_2"])
    print(len(target_df.columns))
    print("original length", len(target_df))
    return target_df, FEATURE_COLS_

## helmet df func

In [14]:
# -------------------------------
# replace polars
# -------------------------------
def target_merge_helmet(target_df, helmet_df, FEATURE_COLS):
    print("original length", len(target_df))
    # set merge-key (game_frame_player_1,2) to merge helmet_df
    target_df = target_df.with_column(((pl.col("step")/10*59.94+5*59.94).cast(int)+1).alias("frame"))
    target_df = target_df.with_column(pl.concat_str([pl.col("game_play"),
                                                     pl.col("frame").cast(str),
                                                     pl.col("nfl_player_id_1"),
                                                    ], sep='_').alias('game_frame_player_1'))
    target_df = target_df.with_column(pl.concat_str([pl.col("game_play"),
                                                     pl.col("frame").cast(str),
                                                     pl.col("nfl_player_id_2"),
                                                    ], sep='_').alias('game_frame_player_2'))
    # set merge key
    helmet_df = helmet_df.with_column(pl.concat_str([pl.col("game_play"),
                                                     pl.col("frame").cast(str),
                                                     pl.col("nfl_player_id"),
                                                    ], sep='_').alias('game_frame_player'))

    # merge target df & helmet_df
    player_views = [[1, "Endzone"],[2, "Endzone"], [1, "Sideline"],[2, "Sideline"]]
    for player_id, view in player_views:
        helmet_view = helmet_df.filter(pl.col("view")==view)
        helmet_view = helmet_view[["game_frame_player", "left", "width", "top", "height"]]
        helmet_view = helmet_view.select(pl.all().suffix(f"_{player_id}"))
        helmet_view = helmet_view.select([pl.col(helmet_view.columns[0]), pl.col(helmet_view.columns[1:]).prefix(f"{view[0]}_")])
        target_df = target_df.join(helmet_view, on=f"game_frame_player_{player_id}", how="left")  
        # add features col
        FEATURE_COLS = add_feature_cols(helmet_view, FEATURE_COLS, [f"game_frame_player_{player_id}"])

    print(len(target_df.columns))
    print("original length", len(target_df))
    return target_df, FEATURE_COLS

---

# Load Target

In [15]:
target_dtypes = {'contact_id':str, 
                'game_play':str,
                'datetime':str,
                'step':int,
                'nfl_player_id_1':str,
                'nfl_player_id_2':str,
                'contact':int,
                }
target_df = pl.read_csv(CFG["TRAIN_LABEL_CSV"], dtypes=target_dtypes)    

FEATURE_COLS = ["nfl_player_id_1", "nfl_player_id_2", "step"]
# display(target_df)

# Merge tracking_df

In [16]:
# -------------------------------
# replace polars
# -------------------------------
tracking_df = pl.read_csv(CFG["TRAIN_TRACKING_CSV"])
tracking_df, SHIFT_COLS = get_tracking_shift(tracking_df)
target_df, FEATURE_COLS = target_merge_tracking(target_df, tracking_df, FEATURE_COLS, SHIFT_COLS)
# display(tracking_df.filter((pl.col("game_play")=="58580_001136") & (pl.col("nfl_player_id").cast(str)=="44830")))# ちゃんとshiftできてそう
del tracking_df

target_df, FEATURE_COLS = create_trackmerged_ftr(target_df, FEATURE_COLS)

SHIFT_FEATURES = []
for col in SHIFT_COLS:
    SHIFT_FEATURES.append(f"{col}_1")
for col in SHIFT_COLS:
    SHIFT_FEATURES.append(f"{col}_2")

219
original length 4721618


# Exclude distance 2

In [17]:
print(len(target_df))
target_df = target_df.filter(pl.col("players_dis") <= 2)
print(len(target_df))

4721618
660553


# Merge helmet df

In [18]:
helmet_df = pl.read_csv(CFG["TRAIN_HELMET_CSV"])
target_df, FEATURE_COLS = target_merge_helmet(target_df, helmet_df, FEATURE_COLS)
target_df, FEATURE_COLS = create_helmetmerged_ftr(target_df, FEATURE_COLS)
target_df, FEATURE_COLS = get_categorical_ftr(target_df, FEATURE_COLS)
print(len(target_df))

original length 660553
252
original length 660553
660553


# Reduce Data

In [19]:
target_df = target_df.to_pandas()

if CFG["DEBUG"]:
    target_df = target_df.sample(CFG["sample_num"]).reset_index(drop=True)
elif CFG["sample_num"] != -1:
    target_df = target_df.sample(CFG["sample_num"]).reset_index(drop=True)

print(len(target_df))
print(len(target_df["game_play"].unique()))
display(target_df["contact"].value_counts())

660553
240


0    596241
1     64312
Name: contact, dtype: int64

In [20]:
target_df[SHIFT_FEATURES]

,x_position_shift-6_1,x_position_shift-5_1,x_position_shift-4_1,x_position_shift-3_1,x_position_shift-2_1,x_position_shift-1_1,x_position_shift0_1,x_position_shift1_1,x_position_shift2_1,x_position_shift3_1,...,sa_shift-4_2,sa_shift-3_2,sa_shift-2_2,sa_shift-1_2,sa_shift0_2,sa_shift1_2,sa_shift2_2,sa_shift3_2,sa_shift4_2,sa_shift5_2
0,40.74,40.65,40.57,40.49,40.43,40.38,40.33,40.30,40.27,40.25,...,3.18,2.84,2.74,2.64,2.43,1.74,1.26,1.00,0.74,0.41
1,40.74,40.65,40.57,40.49,40.43,40.38,40.33,40.30,40.27,40.25,...,2.05,2.61,2.86,2.28,1.64,1.19,0.68,0.47,0.26,0.20
2,40.74,40.65,40.57,40.49,40.43,40.38,40.33,40.30,40.27,40.25,...,0.84,1.28,1.39,1.38,1.54,0.99,0.67,0.30,0.03,-0.08
3,41.50,41.58,41.67,41.78,41.86,41.94,42.00,42.07,42.11,42.17,...,0.84,1.28,1.39,1.38,1.54,0.99,0.67,0.30,0.03,-0.08
4,40.14,40.01,39.90,39.80,39.71,39.64,39.59,39.55,39.53,39.52,...,0.50,1.15,1.73,2.07,2.03,1.71,1.33,1.20,0.53,0.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
660548,34.81,34.54,34.28,34.02,33.75,33.46,33.18,32.92,32.67,32.42,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
660549,40.37,40.03,39.66,39.33,38.96,38.60,38.25,37.88,37.52,37.15,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
660550,24.87,24.66,24.48,24.28,24.07,23.88,23.69,23.49,23.33,23.20,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
660551,62.85,62.82,62.80,62.77,62.72,62.67,62.63,62.58,62.53,62.48,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


# Augmentation

In [21]:
train_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=(-0.1, 0.1), contrast_limit=(-0.3, 0.3), p=0.5),
])

# Dataset

In [22]:
class NFLDataset(Dataset):
    def __init__(self, target_df):
        self.target_df = target_df
        # player 1
        self.xpos_1 = target_df[CFG["xpos_1"]].values
        self.ypos_1 = target_df[CFG["ypos_1"]].values
        self.speed_1 = target_df[CFG["speed_1"]].values
        self.dire_1 = target_df[CFG["dire_1"]].values
        self.orie_1 = target_df[CFG["orie_1"]].values
        self.acc_1 = target_df[CFG["acc_1"]].values
        self.sa_1 = target_df[CFG["sa_1"]].values
        self.dis_1 = target_df[CFG["dis_1"]].values
        # player 2
        self.xpos_2 = target_df[CFG["xpos_2"]].values
        self.ypos_2 = target_df[CFG["ypos_2"]].values
        self.speed_2 = target_df[CFG["speed_2"]].values
        self.dire_2 = target_df[CFG["dire_2"]].values
        self.orie_2 = target_df[CFG["orie_2"]].values
        self.acc_2 = target_df[CFG["acc_2"]].values
        self.sa_2 = target_df[CFG["sa_2"]].values
        self.dis_2 = target_df[CFG["dis_2"]].values

    def __len__(self):
        return len(self.target_df)

    def __getitem__(self, idx):
        target_info = self.target_df.iloc[idx]
        target = target_info.contact
        # player 1
        xpos_1 = self.xpos_1[idx]
        ypos_1 = self.ypos_1[idx]
        speed_1 = self.speed_1[idx]
        dire_1 = self.dire_1[idx]
        orie_1 = self.orie_1[idx]
        acc_1 = self.acc_1[idx]
        sa_1 = self.sa_1[idx]
        # player 2
        xpos_2 = self.xpos_2[idx]
        ypos_2 = self.ypos_2[idx]
        speed_2 = self.speed_2[idx]
        dire_2 = self.dire_2[idx]
        orie_2 = self.orie_2[idx]
        acc_2 = self.acc_2[idx]
        sa_2 = self.sa_2[idx]
        # distance
        dis_1 = self.dis_1[idx]
        dis_2 = self.dis_2[idx]
        
        pos_features = np.concatenate([xpos_1[np.newaxis, :], ypos_1[np.newaxis, :],xpos_2[np.newaxis, :], ypos_2[np.newaxis, :]])
                                        
        speed_features = np.concatenate([speed_1[np.newaxis, :], speed_2[np.newaxis, :]]) 
        dire_features = np.concatenate([dire_1[np.newaxis, :], dire_2[np.newaxis, :]])
        orie_features = np.concatenate([orie_1[np.newaxis, :], orie_2[np.newaxis, :]])
        acc_features = np.concatenate([acc_1[np.newaxis, :], acc_2[np.newaxis, :]])
        sa_features = np.concatenate([sa_1[np.newaxis, :], sa_2[np.newaxis, :]])
        dis_features = np.concatenate([dis_1[np.newaxis, :], dis_2[np.newaxis, :]])
        
        return pos_features, speed_features, dire_features, orie_features, acc_features, sa_features, dis_features, target

In [23]:
if CFG["DEBUG"]:
    check_batch = 4
    pick_df = target_df.copy()
    train_dataset = NFLDataset(pick_df)
    train_loader = DataLoader(
        train_dataset,
        batch_size = check_batch,
        shuffle = False,
        num_workers = CFG["num_workers"],
        pin_memory = True
    )
    for batch_idx, (pos_features, speed_features, dire_features, orie_features, acc_features, sa_features, dis_features, targets) in enumerate(train_loader):
        print("pos_features shape =",pos_features.shape)
        print("speed_features shape =",speed_features.shape)
        print("dire_features shape =",dire_features.shape)
        print("orie_features shape =",orie_features.shape)
        print("acc_features shape =",acc_features.shape)
        print("sa_features shape =",sa_features.shape)
        print("dis_features shape =",dis_features.shape)
        print("target shape", targets.shape)
        break

# Model

In [24]:
class NFLTrackNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.pos_conv = nn.Sequential(
                        nn.Conv1d(4, 1, 5), # x_pos, y_pos
                        nn.Linear(len(CFG["SHIFT_NUM"])-4, CFG["num_track_features"]),
                        nn.ReLU(),
                    )
        self.speed_conv = nn.Sequential(
                        nn.Conv1d(2, 1, 5), # x_pos, y_pos
                        nn.Linear(len(CFG["SHIFT_NUM"])-4, CFG["num_track_features"]),
                        nn.ReLU(),
                    )
        self.dire_conv = nn.Sequential(
                        nn.Conv1d(2, 1, 5), # x_pos, y_pos
                        nn.Linear(len(CFG["SHIFT_NUM"])-4, CFG["num_track_features"]),
                        nn.ReLU(),
                    )

        self.orie_conv = nn.Sequential(
                        nn.Conv1d(2, 1, 5), # x_pos, y_pos
                        nn.Linear(len(CFG["SHIFT_NUM"])-4, CFG["num_track_features"]),
                        nn.ReLU(),
                    )
        self.acc_conv = nn.Sequential(
                        nn.Conv1d(2, 1, 5), # x_pos, y_pos
                        nn.Linear(len(CFG["SHIFT_NUM"])-4, CFG["num_track_features"]),
                        nn.ReLU(),
                    )
        self.sa_conv = nn.Sequential(
                        nn.Conv1d(2, 1, 5), # x_pos, y_pos
                        nn.Linear(len(CFG["SHIFT_NUM"])-4, CFG["num_track_features"]),
                        nn.ReLU(),
                    )
        self.dis_conv = nn.Sequential(
                        nn.Conv1d(2, 1, 5), # x_pos, y_pos
                        nn.Linear(len(CFG["SHIFT_NUM"])-4, CFG["num_track_features"]),
                        nn.ReLU(),
                    )

        self.emb = nn.Linear(CFG["num_track_features"]*7, CFG["num_track_features"])
        self.fc = nn.Linear(CFG["num_track_features"], 1)

    def forward(self, pos, speed, dire, orie, acc, sa, dis):
        pos = self.pos_conv(pos)
        speed = self.speed_conv(speed)
        dire = self.dire_conv(dire)
        orie = self.orie_conv(orie)
        acc = self.acc_conv(acc)
        sa = self.sa_conv(sa)
        dis = self.sa_conv(dis)
        pos = torch.squeeze(pos, dim=1)
        speed = torch.squeeze(speed, dim=1)
        dire = torch.squeeze(dire, dim=1)
        orie = torch.squeeze(orie, dim=1)
        acc = torch.squeeze(acc, dim=1)
        sa = torch.squeeze(sa, dim=1)
        dis = torch.squeeze(dis, dim=1)
               
        embeddings = self.emb(torch.cat([pos, speed, dire, orie, acc, sa, dis], dim=1))
        output = self.fc(embeddings)
        return output, embeddings

# train fn

In [25]:
def train_fn(train_loader, model, criterion, epoch ,optimizer, scheduler):
    model.train()
    batch_time = AverageMeter()
    losses = AverageMeter()
    start, end = time.time(), time.time()
    for batch_idx, (pos_features, speed_features, dire_features, orie_features, acc_features, sa_features, dis_features, targets) in enumerate(train_loader):
        pos_features = pos_features.to(device, non_blocking = True).float()
        speed_features = speed_features.to(device, non_blocking = True).float()
        dire_features = dire_features.to(device, non_blocking = True).float()
        orie_features = orie_features.to(device, non_blocking = True).float()
        acc_features = acc_features.to(device, non_blocking = True).float()
        sa_features = sa_features.to(device, non_blocking = True).float()
        dis_features = dis_features.to(device, non_blocking = True).float()
        targets = targets.to(device, non_blocking = True).float().view(-1, 1)      
        preds, _ = model(pos_features, speed_features, dire_features, orie_features, acc_features, sa_features, dis_features)
        
        loss = criterion(preds, targets)
        losses.update(loss.item(), CFG["batch_size"]) 
        targets = targets.detach().cpu().numpy().ravel().tolist()
        preds = torch.sigmoid(preds).detach().cpu().numpy().ravel().tolist()

        loss.backward() # パラメータの勾配を計算
        optimizer.step() # モデル更新
        optimizer.zero_grad() # 勾配の初期化
                
        batch_time.update(time.time() - end)
        end = time.time()
        if batch_idx % CFG["print_freq"] == 0 or batch_idx == (len(train_loader)-1):
            print('\t Epoch: [{0}][{1}/{2}] '
                    'Elapsed {remain:s} '
                    'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                    .format(
                        epoch, batch_idx, len(train_loader), batch_time=batch_time, loss=losses,
                        remain=timeSince(start, float(batch_idx+1)/len(train_loader)),
            ))
        del preds, pos_features, speed_features, dire_features, orie_features, acc_features, sa_features, dis_features, targets
    gc.collect()
    torch.cuda.empty_cache()
    return losses.avg

# valid fn

In [26]:
def valid_fn(model, valid_loader, criterion):
    model.eval()# モデルを検証モードに設定
    test_targets = []
    test_preds = []
    track_embs = []

    batch_time = AverageMeter()
    losses = AverageMeter()
    start, end = time.time(), time.time()
    view_list = []
    for batch_idx, (pos_features, speed_features, dire_features, orie_features, acc_features, sa_features, dis_features, targets) in enumerate(valid_loader):
        pos_features = pos_features.to(device, non_blocking = True).float()
        speed_features = speed_features.to(device, non_blocking = True).float()
        dire_features = dire_features.to(device, non_blocking = True).float()
        orie_features = orie_features.to(device, non_blocking = True).float()
        acc_features = acc_features.to(device, non_blocking = True).float()
        sa_features = sa_features.to(device, non_blocking = True).float()
        dis_features = dis_features.to(device, non_blocking = True).float()
        targets = targets.to(device, non_blocking = True).float().view(-1, 1)      
        with torch.no_grad():
            preds, track_emb = model(pos_features, speed_features, dire_features, orie_features, acc_features, sa_features, dis_features)
            loss = criterion(preds, targets)
        losses.update(loss.item(), CFG["batch_size"])
        batch_time.update(time.time() - end)

        track_emb = track_emb.detach().cpu().numpy()
        track_embs.extend(track_emb)

        targets = targets.detach().cpu().numpy().ravel().tolist()
        preds = torch.sigmoid(preds).detach().cpu().numpy().ravel().tolist()

        test_preds.extend(preds)
        test_targets.extend(targets)
        # score = matthews_corrcoef(preds, targets)
        if batch_idx % CFG["print_freq"] == 0 or batch_idx == (len(valid_loader)-1):
            print('\t EVAL: [{0}/{1}] '
                'Elapsed {remain:s} '
                'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                .format(
                    batch_idx, len(valid_loader), batch_time=batch_time, loss=losses,
                    remain=timeSince(start, float(batch_idx+1)/len(valid_loader)),
                ))
        del preds, pos_features, speed_features, dire_features, orie_features, acc_features, sa_features, dis_features, targets
        gc.collect()
        torch.cuda.empty_cache()
    test_preds = np.array(test_preds)
    test_targets = np.array(test_targets)
    return test_targets, test_preds, track_embs, losses.avg

# Train loop

In [27]:
def training_loop(target_df):
    # set model & learning fn
    oof_df = pd.DataFrame()
    kf = GroupKFold(n_splits=CFG["n_folds"])
    for fold, (idx_train, idx_valid) in enumerate(kf.split(target_df, target_df["contact_id"], target_df["game_play"])):
        print("---")
        print(f"fold {fold} start training...")
        model = NFLTrackNet()
        model = model.to(device)
        criterion = nn.BCEWithLogitsLoss()
        optimizer = AdamW(model.parameters(), lr=CFG["lr"], weight_decay=CFG["weight_decay"], amsgrad=False)
        scheduler = CosineAnnealingLR(optimizer, T_max=CFG["T_max"], eta_min=CFG["min_lr"], last_epoch=-1)

        if not fold in CFG["train_folds"]:
            print(f"fold{fold} is skip")
            continue
        # separate train/valid data 
        train_df = target_df.iloc[idx_train]
        valid_df = target_df.iloc[idx_valid]
        print("train target contact")
        print(train_df["contact"].value_counts())
        print("valid target contact")
        print(valid_df["contact"].value_counts())
        # separate train/valid data 
        train_dataset = NFLDataset(train_df)
        valid_dataset = NFLDataset(valid_df)
        train_loader = DataLoader(train_dataset,batch_size=CFG["batch_size"], shuffle = True,
                                    num_workers = CFG["num_workers"], pin_memory = True)
        valid_loader = DataLoader(valid_dataset,batch_size=CFG["batch_size"], shuffle = False,
                                    num_workers = CFG["num_workers"], pin_memory = True)

        # training
        best_score = -np.inf
        best_auc = -np.inf
        start_time, end = time.time(), time.time()
        for epoch in range(1, CFG["n_epoch"] + 1):
            print(f'\t === epoch: {epoch}: training ===')
            train_loss_avg = train_fn(train_loader, model, criterion, epoch, optimizer, scheduler)
            valid_targets, valid_preds, valid_embs, valid_loss_avg = valid_fn(model, valid_loader, criterion)

            valid_score = -np.inf
            valid_threshold = 0
            tn_best, fp_best, fn_best, tp_best = 0, 0, 0, 0
            for idx in range(1, 10, 1):
                thr = idx*0.1
                valid_targets = (np.array(valid_targets) > thr).astype(np.int32)
                valid_binary_preds = (np.array(valid_preds) > thr).astype(np.int32)
                score_tmp = matthews_corrcoef(valid_targets, valid_binary_preds)
                cm = confusion_matrix(valid_targets, valid_binary_preds)
                tn, fp, fn, tp = cm.flatten()
                if score_tmp > valid_score:
                    valid_score = score_tmp 
                    valid_threshold = thr
                    tn_best, fp_best, fn_best, tp_best = tn, fp, fn, tp
            elapsed = (time.time() - start_time)/60
            auc_score = roc_auc_score(valid_targets, valid_preds)
            print(f'\t epoch:{epoch}, avg train loss:{train_loss_avg:.4f}, avg valid loss:{valid_loss_avg:.4f}')
            print(f'\t score:{valid_score:.4f}(th={valid_threshold}) AUC = {auc_score:.4f}=> time:{elapsed:.2f} min')
            scheduler.step()
            # validationスコアがbestを更新したらモデルを保存する
            if valid_score > best_score:
                best_score = valid_score
                torch.save(model.state_dict(), f'{CFG["MODEL_DIR"]}/track_fold{fold}.pth')
                print(f'\t Epoch {epoch} - Save Best Score: {best_score:.4f}. Model is saved.')
                contact_id = valid_df["contact_id"].values
                _oof_df = pd.DataFrame({
                    "contact_id" : contact_id,
                    "pred" : valid_preds,
                    "contact" : valid_targets,
                    "fold" : fold,
                })
                track_emb_colname = [f"track_emb_{idx}" for idx in range(CFG["num_track_features"])]
                track_emb_df = pd.DataFrame(valid_embs, columns=track_emb_colname)
                _oof_df = pd.concat([_oof_df, track_emb_df], axis=1)
            
            logging_metrics_epoch(fold, epoch, train_loss_avg, valid_loss_avg, valid_score, valid_threshold, tn_best, fp_best, fn_best, tp_best, auc_score)

        del train_loader, train_dataset, valid_loader, valid_dataset
        oof_df = pd.concat([oof_df, _oof_df], axis = 0)
        del _oof_df
        gc.collect()
        torch.cuda.empty_cache()
    return oof_df

In [28]:
if CFG["kaggle"]:
    oof_df = training_loop(target_df)
    wandb.finish()
else:
    with mlflow.start_run(experiment_id=experiment_id, run_name=CFG["EXP_NAME"]) as run:
        mlflow.log_dict(CFG, "configuration.yaml")
        mlflow.log_param("positive data num", len(target_df[target_df["contact"]==1]))
        mlflow.log_param("negative data num", len(target_df[target_df["contact"]==0]))
        oof_df = training_loop(target_df)

---
fold 0 start training...
train target contact
0    397742
1     42603
Name: contact, dtype: int64
valid target contact
0    198499
1     21709
Name: contact, dtype: int64
	 === epoch: 1: training ===
	 Epoch: [1][0/1721] Elapsed 0m 0s (remain 7m 7s) Loss: 1.5059(1.5059) 
	 Epoch: [1][1000/1721] Elapsed 0m 31s (remain 0m 22s) Loss: 0.2760(0.2610) 
	 Epoch: [1][1720/1721] Elapsed 0m 52s (remain 0m 0s) Loss: 0.2308(0.2548) 
	 EVAL: [0/861] Elapsed 0m 0s (remain 1m 50s) Loss: 0.1582(0.1582) 
	 EVAL: [860/861] Elapsed 1m 10s (remain 0m 0s) Loss: 0.2164(0.2479) 
	 epoch:1, avg train loss:0.2548, avg valid loss:0.2479
	 score:0.3614(th=0.2) AUC = 0.8512=> time:2.07 min
	 Epoch 1 - Save Best Score: 0.3614. Model is saved.
	 === epoch: 2: training ===
	 Epoch: [2][0/1721] Elapsed 0m 0s (remain 5m 37s) Loss: 0.2444(0.2444) 
	 Epoch: [2][1000/1721] Elapsed 0m 29s (remain 0m 21s) Loss: 0.2404(0.2426) 
	 Epoch: [2][1720/1721] Elapsed 0m 50s (remain 0m 0s) Loss: 0.1619(0.2428) 
	 EVAL: [0/861] E

# Save oof_df

In [29]:
display(oof_df)
if CFG["kaggle"]:
    oof_filename = os.path.join(CFG["OUTPUT_DIR"], "oof_df.csv")
    oof_df.to_csv(oof_filename, index=False)
else:
    oof_filename = os.path.join(CFG["OUTPUT_DIR"], CFG["EXP_NAME"], "oof_df.csv")
    oof_df.to_csv(oof_filename, index=False)

,contact_id,pred,contact,fold,track_emb_0,track_emb_1,track_emb_2
0,58168_003392_0_38590_44822,0.043566,0,0,-8.036210,-5.183720,-1.396353
1,58168_003392_0_38590_39947,0.044455,0,0,-7.164643,-5.979579,-3.174212
2,58168_003392_0_38590_42565,0.045948,0,0,-6.898588,-6.107840,-3.242111
3,58168_003392_0_41944_42565,0.106005,0,0,-2.492723,-6.486005,-1.608166
4,58168_003392_0_37211_46445,0.059035,0,0,-6.365085,-5.183343,-2.545109
...,...,...,...,...,...,...,...
220067,58582_003121_91_48220_G,0.002271,0,2,-16.712540,13.505566,0.059685
220068,58582_003121_91_47906_G,0.000738,0,2,-20.175674,15.787048,0.165789
220069,58582_003121_91_38557_G,0.003363,0,2,-15.320747,13.059745,-0.106152
220070,58582_003121_91_47872_G,0.042308,0,2,-8.127855,6.526262,-0.230339


In [30]:
for idx in range(1, 10, 1):
    thr = idx*0.1
    valid_targets = (np.array(oof_df["contact"]) > thr).astype(np.int32)
    valid_binary_preds = (np.array(oof_df["pred"]) > thr).astype(np.int32)
    score = matthews_corrcoef(valid_targets, valid_binary_preds)
    print(f"threshold={thr:.3f}, score={score:.5f}")

threshold=0.100, score=0.34584
threshold=0.200, score=0.36136
threshold=0.300, score=0.32480
threshold=0.400, score=0.24315
threshold=0.500, score=0.16338
threshold=0.600, score=0.09314
threshold=0.700, score=0.04484
threshold=0.800, score=0.02067
threshold=0.900, score=0.00571


In [31]:
alldata_df = pd.read_csv("/workspace/input/train_labels.csv")
alldata_df = alldata_df.merge(oof_df[["contact_id", "pred"]], on="contact_id", how="left")

In [32]:
for idx in range(1, 10, 1):
    thr = idx*0.1
    valid_targets = (np.array(alldata_df["contact"]) > thr).astype(np.int32)
    valid_binary_preds = (np.array(alldata_df["pred"]) > thr).astype(np.int32)
    score = matthews_corrcoef(valid_targets, valid_binary_preds)
    print(f"threshold={thr:.3f}, score={score:.5f}")

threshold=0.100, score=0.43634
threshold=0.200, score=0.43214
threshold=0.300, score=0.37858
threshold=0.400, score=0.28185
threshold=0.500, score=0.18824
threshold=0.600, score=0.10768
threshold=0.700, score=0.05222
threshold=0.800, score=0.02378
threshold=0.900, score=0.00644
